In [64]:
from tqdm import tqdm

In [18]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

#  セッションを開いて、変数を初期化してfを評価する
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
sess.close()

# with ブロックを使用
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

# initで全ての変数を初期化する
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

    # InteracticeSession を作ると自動的にデフォルトセッションになる
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [19]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [21]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


In [23]:
w = tf.constant(3)
x =w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())
    
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y.eval())
    print(z.eval())
 

10
15
10
15


In [32]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n =housing.data.shape

housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()


9

In [55]:
from sklearn.preprocessing import StandardScaler
n_epochs = 1000
learning_rate = 0.01

sc = StandardScaler()
scaled_housing_data_plus_bias = sc.fit_transform(housing_data_plus_bias)
print(housing.target)

X = tf.constant(scaled_housing_data_plus_bias, dtype =tf.float32, name ='X')
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name='y' )

theta = tf.Variable(tf.random_uniform([n+1,1],-1.0, 1.0), name = 'theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse')
# 勾配の計算
#gradients = 2/m*tf.matmul(tf.transpose(X), error)
#gradients = tf.gradients(mse,[theta])[0]

# 変数の値を更新　
#training_op = tf.assign(theta, theta - learning_rate * gradients)

# Tensor Flow に実装されているオプティマイザを使う場合
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate) #SGD
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

training_op = optimizer.minimize(mse)


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch %100 == 0:
            print('Epoch',epoch, 'MSE = ',mse.eval())
        sess.run(training_op)

        best_theta = theta.eval()

[4.526 3.585 3.521 ... 0.923 0.847 0.894]
Epoch 0 MSE =  7.571035
Epoch 100 MSE =  4.809864
Epoch 200 MSE =  4.803888
Epoch 300 MSE =  4.803334
Epoch 400 MSE =  4.8032646
Epoch 500 MSE =  4.803255
Epoch 600 MSE =  4.8032546
Epoch 700 MSE =  4.8032537
Epoch 800 MSE =  4.8032537
Epoch 900 MSE =  4.8032546


In [56]:
A = tf.placeholder(tf.float32, shape=(None,3))
B = A + 3
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})

print(B_val_1)
print(B_val_2)

[[4. 5. 6.]]
[[ 7.  8.  9.]
 [10. 11. 12.]]


In [57]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch


In [81]:
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None,n+1),name='X')
y = tf.placeholder(tf.float32, shape=(None,1), name='y')

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
saver2 = tf.train.Saver({'weight':theta})

with tf.Session() as sess:
    sess.run(init)

    for epoch in tqdm(range(n_epochs)):
        if epoch % 100 ==0:
            save_path = saver.save(sess,'tmp/my_model.ckpt')
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
    save_path = saver.save(sess,'tmp/my_model_final.index')
    

100%|██████████| 1000/1000 [01:17<00:00, 12.97it/s]


In [80]:
with tf.Session() as sess:
    saver.restore(sess, 'tmp/my_model_final.ckpt')
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from tmp/my_model_final.ckpt


In [78]:
saver = tf.train.import_meta_graph('tmp/my_model_final.ckpt.meta')
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess,'tmp/my_model_final.ckpt')